# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 33

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [8]:
#metric = 'valuePoints metric'
metric = 'adjusted points per game'
#metric = 'form 10'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
340,Lloris,Tottenham,53,16.086957,74,73.347764,4.6,4.559456,5.028447,5.092690,1.980501
411,Patrício,Wolves,53,32.926829,135,144.224646,4.1,4.380156,3.735636,4.214820,1.902618
471,Henderson,Sheffield Utd,52,31.111111,140,129.077990,4.5,4.148935,4.200136,4.393053,1.819429
168,Schmeichel,Leicester City,54,33.255814,143,137.054436,4.3,4.121217,5.467374,5.539427,1.773490
93,Pope,Burnley,50,33.111111,149,136.345638,4.5,4.117821,4.576530,4.612115,1.841546
189,Alisson,Liverpool,62,23.750000,114,94.308907,4.8,3.970901,4.155421,3.883255,1.594752
235,de Gea,Manchester Utd,53,32.777778,118,128.323360,3.6,3.914950,3.908139,4.044068,1.700546
131,Guaita,Crystal Palace,49,30.000000,126,117.345399,4.2,3.911513,3.231023,4.118845,1.767042
14,Leno,Arsenal,48,30.000000,114,114.708370,3.8,3.823612,4.693690,4.227914,1.745232


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
182,Alexander-Arnold,Liverpool,78,33.275862,193,176.377267,5.8,5.300457,4.041179,4.493280,1.897868
103,Alonso,Chelsea,62,13.965517,81,66.590610,5.8,4.768216,3.283753,4.695172,1.914962
401,Doherty,Wolves,64,31.086957,143,147.630785,4.6,4.748962,5.012376,5.584971,1.877192
181,Robertson,Liverpool,70,30.816327,151,142.938640,4.9,4.638406,2.277191,3.137790,1.753153
105,Azpilicueta,Chelsea,60,30.810811,114,141.718861,3.7,4.599647,6.042975,5.275862,1.877798
297,Lundstram,Sheffield Utd,46,29.777778,134,128.615324,4.5,4.319171,2.552726,3.345190,2.013824
183,van Dijk,Liverpool,65,33.265306,163,138.602013,4.9,4.166564,3.825323,3.851604,1.634261
159,Pereira,Leicester City,62,27.954545,123,113.116439,4.4,4.046442,4.579843,4.315085,1.625090
122,Wan-Bissaka,Manchester Utd,55,30.277778,109,121.138608,3.6,4.000908,5.122194,4.938767,1.705993


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,126,28.873239,205,193.116808,7.1,6.688436,5.392535,5.369456,1.884253
618,Fernandes,Manchester Utd,89,8.988764,80,58.156926,8.9,6.469958,6.983216,7.000370,2.168735
215,De Bruyne,Manchester City,108,30.857143,216,194.715174,7.0,6.310214,6.892172,6.747453,1.920137
192,Mané,Liverpool,124,29.850746,200,177.513801,6.7,5.946712,5.699634,5.188991,1.688753
214,Sterling,Manchester City,117,28.269231,147,158.291000,5.2,5.599410,5.565203,5.738716,1.637002
342,Son,Tottenham,98,24.821429,139,124.687266,5.6,5.023372,4.542235,4.969071,1.604657
239,Martial,Manchester Utd,83,27.000000,162,131.750038,6.0,4.879631,6.101347,5.545550,1.693745
431,Pulisic,Chelsea,72,20.000000,98,96.837794,4.9,4.841890,6.176452,6.228629,1.804466
217,Mahrez,Manchester City,86,27.777778,150,130.409881,5.4,4.694756,3.737749,4.238360,1.600898


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
233,Rashford,Manchester Utd,91,25.932203,153,146.953573,5.9,5.666837,5.366839,5.356533,1.878538
166,Vardy,Leicester City,97,30.000000,186,162.080206,6.2,5.402674,5.937950,4.989007,1.734694
11,Aubameyang,Arsenal,109,30.877193,176,153.673470,5.7,4.976925,5.709019,5.399332,1.507467
409,Jiménez,Wolves,83,32.941176,168,163.840206,5.1,4.973721,3.823284,4.223472,1.726404
210,Agüero,Manchester City,117,24.000000,132,117.140206,5.5,4.880842,4.967317,4.281874,1.426927
187,Firmino,Liverpool,94,32.619048,137,153.606941,4.2,4.709118,2.553267,3.271460,1.535945
460,Abraham,Chelsea,74,28.958333,139,134.780206,4.8,4.654280,3.103713,3.226058,1.710948
313,Ings,Southampton,73,32.941176,168,148.426735,5.1,4.505812,4.468336,3.978403,1.667676
278,Pukki,Norwich City,64,32.195122,132,139.280103,4.1,4.326124,1.638308,2.302100,1.710051


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
metric = 'adjusted points per game'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
191,Salah,Liverpool,126,28.873239,205,193.116808,7.1,6.688436,5.392535,5.369456,1.884253
618,Fernandes,Manchester Utd,89,8.988764,80,58.156926,8.9,6.469958,6.983216,7.000370,2.168735
215,De Bruyne,Manchester City,108,30.857143,216,194.715174,7.0,6.310214,6.892172,6.747453,1.920137
192,Mané,Liverpool,124,29.850746,200,177.513801,6.7,5.946712,5.699634,5.188991,1.688753
233,Rashford,Manchester Utd,91,25.932203,153,146.953573,5.9,5.666837,5.366839,5.356533,1.878538
214,Sterling,Manchester City,117,28.269231,147,158.291000,5.2,5.599410,5.565203,5.738716,1.637002
166,Vardy,Leicester City,97,30.000000,186,162.080206,6.2,5.402674,5.937950,4.989007,1.734694
182,Alexander-Arnold,Liverpool,78,33.275862,193,176.377267,5.8,5.300457,4.041179,4.493280,1.897868
342,Son,Tottenham,98,24.821429,139,124.687266,5.6,5.023372,4.542235,4.969071,1.604657


In [10]:
metric = 'form 10'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
618,Fernandes,Manchester Utd,89,8.988764,80,58.156926,8.9,6.469958,6.983216,7.000370,2.168735
215,De Bruyne,Manchester City,108,30.857143,216,194.715174,7.0,6.310214,6.892172,6.747453,1.920137
257,Lejeune,Newcastle Utd,43,6.000000,24,17.774597,4.0,2.962433,NaN,6.335912,1.428612
431,Pulisic,Chelsea,72,20.000000,98,96.837794,4.9,4.841890,6.176452,6.228629,1.804466
113,Willian,Chelsea,72,32.173913,148,139.496327,4.6,4.335697,8.182119,6.213671,1.615819
389,Antonio,West Ham,69,19.142857,67,81.385602,3.5,4.251487,6.628707,5.855884,1.618513
214,Sterling,Manchester City,117,28.269231,147,158.291000,5.2,5.599410,5.565203,5.738716,1.637002
401,Doherty,Wolves,64,31.086957,143,147.630785,4.6,4.748962,5.012376,5.584971,1.877192
239,Martial,Manchester Utd,83,27.000000,162,131.750038,6.0,4.879631,6.101347,5.545550,1.693745


In [11]:
metric = 'valuePoints metric'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
618,Fernandes,Manchester Utd,89,8.988764,80,58.156926,8.9,6.469958,6.983216,7.000370,2.168735
297,Lundstram,Sheffield Utd,46,29.777778,134,128.615324,4.5,4.319171,2.552726,3.345190,2.013824
340,Lloris,Tottenham,53,16.086957,74,73.347764,4.6,4.559456,5.028447,5.092690,1.980501
215,De Bruyne,Manchester City,108,30.857143,216,194.715174,7.0,6.310214,6.892172,6.747453,1.920137
103,Alonso,Chelsea,62,13.965517,81,66.590610,5.8,4.768216,3.283753,4.695172,1.914962
411,Patrício,Wolves,53,32.926829,135,144.224646,4.1,4.380156,3.735636,4.214820,1.902618
182,Alexander-Arnold,Liverpool,78,33.275862,193,176.377267,5.8,5.300457,4.041179,4.493280,1.897868
191,Salah,Liverpool,126,28.873239,205,193.116808,7.1,6.688436,5.392535,5.369456,1.884253
233,Rashford,Manchester Utd,91,25.932203,153,146.953573,5.9,5.666837,5.366839,5.356533,1.878538
